In [ ]:
import urllib
from urllib.request import urlopen, urlretrieve
from urllib.parse import quote_plus
from bs4 import BeautifulSoup
import regex as re
from tqdm.notebook import tqdm
import os

site = 'http://www.polosedan-club.com/'
folder_names = [
    ['0', 'Отзывы', 'Отзывы-владельцев-о-Поло-седан-polo-sedan.13', 'FW POLO'],
    ['1', 'Двигатель', 'Двигатель-и-его-системы.18', 'FW POLO'],
    ['2', 'Трансмиссия', 'Сцепление-и-коробка-передач.19', 'FW POLO'],
    ['3', 'Ходовая', 'Ходовая-и-подвеска.20', 'FW POLO'],
    ['4', 'Колеса', 'Шины-и-колесные-диски.32', 'FW POLO'],
    ['5', 'Электрооборудование', 'Электрооборудование.21', 'FW POLO'],
    ['6', 'Мультимедиа', 'Аудио-Видео-системы-автомобиля.43', 'FW POLO'],
    ['7', 'Кузов', 'Кузов-и-система-отопления.22', 'FW POLO'],
    ['8', 'Жидкости', 'Топливо-и-эксплуатационные-жидкости.23', 'FW POLO'],
    ['9', 'Сервис', 'Гарантийное-и-сервисное-обслуживание.16', 'FW POLO'],
    ['10', 'Эксплуатация', 'Общие-вопросы-эксплуатации.17', 'FW POLO'],
    ['11', 'Тюнинг', 'Стайлинг-Тюнинг-Доп-оборудование-Защита.24', 'FW POLO'],
    ['12', 'Эксплуатация-Ремонт', 'Ремонт-и-доработки-своими-силами.48', 'FW POLO'],
    ['13', 'Эксплуатация-Вопросы', 'Вопросы-возникшие-при-эксплуатации-автомобиля.47', 'FW POLO'],
    ['14', 'Проблемы-Заводской брак', 'Проблемы-заводской-брак-и-тд.44', 'FW POLO']
]

На странице раздела у нас две задачи:  
1. Найти и сохранить все ссылки на темы
2. Найти ссылку на следующую страницу раздела или убедиться, что это последняя страница раздела.

In [ ]:
def collect_themes(site, folder):
    
    site_address = site + 'forums/' + quote_plus(folder) + '/'
    themes = []
    flag = True

    while flag:
    
        resp = urlopen(site_address) # скачиваем файл
        html = resp.read().decode('utf-8') # считываем содержимое
        soup = BeautifulSoup(html, 'html.parser') # делаем суп
    
        flag = False
        for li in soup.find_all('li'):
            if li.find('h3'):
                h3 = li.find('h3')
                if h3.find('a'):
                    link = h3.find('a')
                    s = link.get('href')
                    theme = site + s
                    themes.append([link.get_text(), theme])
        
        for link in soup.find_all('a'):
            if link.has_attr('href'):
                if link.has_attr('class') and link.get('class') == ['text']:
                    if re.search('Вперёд', link.get_text()):
                        flag = True
                        s = link.get('href')
                        site_address = site + s 
                        break
    return themes

На странице темы у нас три задачи:  
1. Понять, что являеся шапкой темы (это сообщение #1), прочитать ее только один раз - НЕ СДЕЛАНО
2. Прочитать и сохранить все сообщения (кроме шапки) на странице темы
3. Найти ссылку на следующую страницу темы или убедиться, что это последняя страница темы.

In [ ]:
def collect_messages(site, themes):

    messages = []
    for theme in tqdm(themes):
        
        print(theme[0])
        theme_address = theme[1]
        flag = True
        theme_messages = []
        post_data = {}

        while flag:
            try:
                resp = urlopen(theme_address) # скачиваем файл
            except:
                break
            html = resp.read().decode('utf-8', errors='ignore') # считываем содержимое
            soup = BeautifulSoup(html, 'html.parser') # делаем суп
            #Собираем сами сообщения
            lis = soup.find_all('li')
            for li in lis:
                if li.has_attr('id') and re.search('post-', li['id']):
                    author = li['data-author']
                    for div in li.find_all('div'):
                        if div.has_attr('class') and div['class'] == ['messageContent']:
                            blockquote = div.find('article').find('blockquote')
                            if blockquote.find('div'):
                                quote = blockquote.find('div')
                                quote.clear()
                            s = blockquote.get_text().strip()
                            s = re.sub('[\']+', '`', s)
                            s = '\'' + re.sub('[\s]+', ' ', s).strip() + '\''
                            post_date = ''
                            more_divs = div.find_next_siblings('div')
                            for next_div in more_divs:
                                for span in next_div.find_all('span'):
                                    if re.search('[0-9]+ [а-я]+ [0-9]{4}', span.get_text().strip()):
                                        post_date = re.search('[0-9]+ [а-я]+ [0-9]{4}', span.get_text().strip())[0]
                                        break
                            if len(s) > 2:
                                theme_messages.append([post_date, author, s])
            # В отдельном цикле ищем ссылку на следующую страницу темы (это 'a' с атрибутом 'rel', равным 'next')
            # Если ссылка найдена, 
            flag = False 
            for link in soup.find_all('a'):
                if link.has_attr('href'):
                    if link.has_attr('class') and link.get('class') == ['text']:
                        if re.search('Вперёд', link.get_text()):
                            flag = True
                            s = link.get('href')
                            theme_address = site + s
                            break
        if len(theme_messages) > 0:
            messages.append(['\''+ theme[0] + '\'', theme_messages])
        
    return messages

In [ ]:
def save_messages(messages, folder_name):
    f_name = os.path.join(folder_name[3], folder_name[3] + '_' + folder_name[0] + '_' + folder_name[1] + '.csv')
    with open(f_name, 'w', encoding='utf-8') as ouf:
        ouf.write('Code,Folder,Theme,Date,Author,Message\n')
        for message in messages:
            header = folder_name[:2] + [message[0]]
            for item in message[1]:
                ouf.write(','.join(header + item))
                ouf.write('\n')

In [ ]:
def scraper(site, folder_names):
    for folder_name in folder_names:
        if len(folder_name[2]) > 0:
            folder = folder_name[2]
            current_themes = collect_themes(site, folder)
            current_messages = collect_messages(site, current_themes)
            save_messages(current_messages, folder_name)
    print('Information collected')

In [ ]:
scraper(site, folder_names)

In [ ]:
#import pandas as pd
#mess = pd.read_csv('./FW POLO/FW POLO_5_Электрооборудование.csv', quotechar="'")
#mess